In [1]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from pyspark.sql import functions as Func
from pyspark.sql.functions import upper,col,sum,avg,max

sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 0.37.0 
Authenticating with environment variables and user-defined glue_role_arn: arn:aws:iam::214541855063:role/AWSGlueServiceRole-Lab4
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 5
Session ID: 58045d95-c909-4bd7-8454-49fe1a178023
Job Type: glueetl
Applying the following default arguments:
--glue_kernel_version 0.37.0
--enable-glue-datacatalog true
Waiting for session 58045d95-c909-4bd7-8454-49fe1a178023 to get into ready status...
Session 58045d95-c909-4bd7-8454-49fe1a178023 has been created.




- Na célula a seguir, adicione o código necessário para ler o conteúdo do arquivo _nomes.csv_ para um dataframe Spark.



In [2]:
nomes = spark.read.load("s3://azimuth.com/lab-glue/input/nomes.csv", header=True, format="csv", sep=",", inferSchema=True)

- Imprima o esquema do dataframe

In [3]:
nomes.schema

StructType(List(StructField(nome,StringType,true),StructField(sexo,StringType,true),StructField(total,IntegerType,true),StructField(ano,IntegerType,true)))


- Na próxima célula, escreva o código necessário para alterar a caixa dos valores da coluna _nome_ para MAIÚSCULO

In [4]:
nomes.select("*", upper(col("nome"))).show()


+---------+----+-----+----+-----------+
|     nome|sexo|total| ano|upper(nome)|
+---------+----+-----+----+-----------+
| Jennifer|   F|54336|1983|   JENNIFER|
|  Jessica|   F|45278|1983|    JESSICA|
|   Amanda|   F|33752|1983|     AMANDA|
|   Ashley|   F|33292|1983|     ASHLEY|
|    Sarah|   F|27228|1983|      SARAH|
|  Melissa|   F|23472|1983|    MELISSA|
|   Nicole|   F|22392|1983|     NICOLE|
|Stephanie|   F|22323|1983|  STEPHANIE|
|  Heather|   F|20749|1983|    HEATHER|
|Elizabeth|   F|19838|1983|  ELIZABETH|
|  Crystal|   F|17904|1983|    CRYSTAL|
|      Amy|   F|17095|1983|        AMY|
| Michelle|   F|16828|1983|   MICHELLE|
|  Tiffany|   F|15960|1983|    TIFFANY|
| Kimberly|   F|15374|1983|   KIMBERLY|
|Christina|   F|15359|1983|  CHRISTINA|
|    Amber|   F|14886|1983|      AMBER|
|     Erin|   F|14835|1983|       ERIN|
|  Rebecca|   F|14711|1983|    REBECCA|
|   Rachel|   F|14588|1983|     RACHEL|
+---------+----+-----+----+-----------+
only showing top 20 rows


- Apresente a contagem de linhas contidas no dataframe

In [5]:
nomes.count()

1825433


- Agora, apresente a contagem de nomes, agrupando os dados do dataframe pelas colunas _ano_ e _sexo_. Ordene os dados de modo que o ano mais recente aparece como primeiro registro do dataframe.

In [6]:
nomes.groupBy("ano","sexo").agg(sum("total")).orderBy(Func.col("ano").desc()).show()

+----+----+----------+
| ano|sexo|sum(total)|
+----+----+----------+
|2014|   M|   1901376|
|2014|   F|   1768775|
|2013|   F|   1745339|
|2013|   M|   1881463|
|2012|   F|   1753922|
|2012|   M|   1889414|
|2011|   M|   1893230|
|2011|   F|   1753500|
|2010|   F|   1772738|
|2010|   M|   1913851|
|2009|   M|   1979303|
|2009|   F|   1832925|
|2008|   M|   2036289|
|2008|   F|   1887234|
|2007|   M|   2072139|
|2007|   F|   1919408|
|2006|   M|   2052377|
|2006|   F|   1898463|
|2005|   F|   1845379|
|2005|   M|   1994841|
+----+----+----------+
only showing top 20 rows


- Qual foi o nome feminino com mais registro e em que ano?

In [7]:
nomes.filter(Func.col("sexo") == "F").orderBy(Func.col("total").desc()).show(1)

+-----+----+-----+----+
| nome|sexo|total| ano|
+-----+----+-----+----+
|Linda|   F|99680|1947|
+-----+----+-----+----+
only showing top 1 row


- Qual foi o nome masculino com mais registro e em que ano?

In [8]:
nomes.filter(Func.col("sexo") == "M").orderBy(Func.col("total").desc()).show(1)

+-----+----+-----+----+
| nome|sexo|total| ano|
+-----+----+-----+----+
|James|   M|94755|1947|
+-----+----+-----+----+
only showing top 1 row


- Apresente o total de registros (masculinos e femininos) para cada ano presente no dataframe. Considere apenas as primeiras 10 linhas, ordenadas pelo ano, de forma crescente.

In [9]:
nomes.groupBy("ano").agg(sum("total")).orderBy(Func.col("ano").desc()).show(10)

+----+----------+
| ano|sum(total)|
+----+----------+
|2014|   3670151|
|2013|   3626802|
|2012|   3643336|
|2011|   3646730|
|2010|   3686589|
|2009|   3812228|
|2008|   3923523|
|2007|   3991547|
|2006|   3950840|
|2005|   3840220|
+----+----------+
only showing top 10 rows


- Escreva o conteúdo do dataframe com os valores de nome em maiúsculo no S3. Atenção aos requisitos:
  - A gravação deve ocorrer no subdiretório _frequencia_registro_nomes_eua_ do path s3://\<BUCKET\>/lab-glue/
  - O formato deve ser JSON
  - O particionamento deverá ser realizado pelas colunas _sexo_ e _ano_ (nesta ordem)

In [15]:
dfnomes = nomes.select("*", upper(col("nome")))
dfnomes.write.partitionBy("sexo","ano").format("json").save("s3://azimuth.com/lab-glue/frequencia_registro_nomes_eua/")

In [16]:
job.commit()